# atmodeller

## Tutorial 1: basic operation

Import the required functionality.

In [ ]:
from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import SystemConstraints, FugacityConstraint, BufferedFugacityConstraint, ElementMassConstraint
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem
from atmodeller.core import GasSpecies, Species
from atmodeller.solubility.carbon_species import CO2_basalt_dixon
from atmodeller.solubility.hydrogen_species import H2O_peridotite_sossi
from atmodeller.utilities import earth_oceans_to_hydrogen_mass
import logging

There are two loggers available: by default, `debug_logger()` will send DEBUG output to the terminal, whereas `debug_file_logger()` will send INFO output to the terminal and DEBUG output to a file.

In [ ]:
# logger = debug_file_logger()
logger = debug_logger()

You can change the logging level to reduce the logging output:

In [ ]:
logger.setLevel(logging.INFO)

### 1. Simple H2O-H2-O2 system

We define a list of the species we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere via a solubility law:

In [ ]:
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")

species = Species([H2O_g, H2_g, O2_g])

We can then create an interior-atmosphere system using the list of species and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in another notebook.

In [ ]:
interior_atmosphere = InteriorAtmosphereSystem(species=species)

To solve the system, we provide a constraint of the H2O and O2 fugacity in bar. Here, the O2 fugacity is constrained at the IW buffer:

In [ ]:
H2O_fugacity  = FugacityConstraint(H2O_g, 1)
O2_fugacity = BufferedFugacityConstraint(O2_g, IronWustiteBuffer())

constraints = SystemConstraints([H2O_fugacity, O2_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)

You can perform a quick check on the solution directly using:

In [ ]:
interior_atmosphere.output_solution()

You can access all the output quantities in a dictionary, although this is usually not the most convenient way of working with the output:

In [ ]:
interior_atmosphere.output()

It's often convenient to look at the output in Excel. The following exports an Excel file:

In [ ]:
interior_atmosphere.output(file_prefix='tutorial1_basics', to_excel=True)

Or alternatively you can access a dictionary of dataframes for analysis:

In [ ]:
dataframe_dict = interior_atmosphere.output(to_dataframes=True)
dataframe_dict

### 2. System with C and H and prescribed pressures

We now extend the species list to additionally include C-species:

In [ ]:
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2", solubility=CO2_basalt_dixon())

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g])

In [ ]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity, which below is shifted by three log10 units relative to the iron-wustite buffer:

In [ ]:
H2O_fugacity = FugacityConstraint(H2O_g, 1)
CO2_fugacity = FugacityConstraint(CO2_g, 1)
O2_fugacity = BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=3))

constraints = SystemConstraints([H2O_fugacity, CO2_fugacity, O2_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.output_solution()

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three fugacity constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution.

In [ ]:
H2O_fugacity = FugacityConstraint(H2O_g, 1)
H2_fugacity = FugacityConstraint(H2_g, 1)
CO_fugacity = FugacityConstraint(CO_g, 1)

constraints = SystemConstraints([H2O_fugacity, H2_fugacity, CO_fugacity])

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.output_solution()

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of fugacity and mass constraints. We define the same species as before:

In [ ]:
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2", solubility=CO2_basalt_dixon())

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g])

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [ ]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_hydrogen_mass(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    ElementMassConstraint("H", mass_H),
    ElementMassConstraint("C", mass_C),
    BufferedFugacityConstraint(O2_g, IronWustiteBuffer())
])

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints, factor=1)
interior_atmosphere.output_solution()

### 4. Including more species such as CH4

It is straightforward to add more species to the system:

In [ ]:
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2", solubility=CO2_basalt_dixon())
CH4_g = GasSpecies("CH4")

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g])

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [ ]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_hydrogen_mass(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    ElementMassConstraint("H", mass_H),
    ElementMassConstraint("C", mass_C),
    BufferedFugacityConstraint(O2_g, IronWustiteBuffer())
])

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints)
interior_atmosphere.output_solution()